In [1]:
!pip install pyspark==3.3.2 py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824006 sha256=fc25228fd9f7fbd711686f5edd4411e26b3192adea9d1d996ed0c6be7090fe9f
  Stored in directory: /root/.cache/pip/wheels/47/69/84/c7c7776e2287a654536f5cba7dc54c904c03aa2c3e29206f0f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-conne

In [10]:
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

# Настройка Spark

from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("weather").getOrCreate()

**Question 1: Install Spark and PySpark**

In [3]:
spark.version

'3.3.2'

In [4]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-04 19:16:23--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.200, 54.230.209.126, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  67.1MB/s    in 0.9s    

2025-03-04 19:16:24 (67.1 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [12]:
df.show(10, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|2       |2024-10-01 00:30:44 |2024-10-01 00:48:26  |1              |3.0          |1         |N                 |162         |246         |1           |18.4       |1.0  |0.5    |1.5      

In [13]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



**Question 2: Yellow October 2024**

In [7]:
df.repartition(4).write.parquet('./yellow_repartitioned')

In [32]:
!du -sh ./yellow_repartitioned/*

25M	./yellow_repartitioned/part-00000-3d58d84c-84b5-4af0-a17a-59aa1dddd082-c000.snappy.parquet
25M	./yellow_repartitioned/part-00001-3d58d84c-84b5-4af0-a17a-59aa1dddd082-c000.snappy.parquet
25M	./yellow_repartitioned/part-00002-3d58d84c-84b5-4af0-a17a-59aa1dddd082-c000.snappy.parquet
25M	./yellow_repartitioned/part-00003-3d58d84c-84b5-4af0-a17a-59aa1dddd082-c000.snappy.parquet
0	./yellow_repartitioned/_SUCCESS


**Question 3: Count records**

In [28]:
df.withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
.filter("pickup_date='2024-10-15'") \
.count()

128893

**Question 4: Longest trip**

In [31]:
df.withColumn("diff_hours", ((F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long")) / 3600).cast("int")) \
.select('tpep_pickup_datetime','tpep_dropoff_datetime','diff_hours') \
.agg(F.max(F.col('diff_hours'))).collect()

[Row(max(diff_hours)=162)]

**Question 5: User Interface**

In [34]:
4040

4040

In [35]:
! wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-04 20:15:38--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.126, 54.230.209.140, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.01s   

2025-03-04 20:15:38 (843 KB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [38]:
zones = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')
zones.show(5, False)
zones.printSchema()

+----------+-------------+-----------------------+------------+
|LocationID|Borough      |Zone                   |service_zone|
+----------+-------------+-----------------------+------------+
|1         |EWR          |Newark Airport         |EWR         |
|2         |Queens       |Jamaica Bay            |Boro Zone   |
|3         |Bronx        |Allerton/Pelham Gardens|Boro Zone   |
|4         |Manhattan    |Alphabet City          |Yellow Zone |
|5         |Staten Island|Arden Heights          |Boro Zone   |
+----------+-------------+-----------------------+------------+
only showing top 5 rows

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



**Question 6: Least frequent pickup location zone**

In [47]:
df.join(zones, df.PULocationID == zones.LocationID, how='inner') \
  .select('Zone','tpep_pickup_datetime') \
  .groupBy('Zone') \
  .count() \
  .orderBy('count') \
  .show(5, False)

+---------------------------------------------+-----+
|Zone                                         |count|
+---------------------------------------------+-----+
|Governor's Island/Ellis Island/Liberty Island|1    |
|Rikers Island                                |2    |
|Arden Heights                                |2    |
|Green-Wood Cemetery                          |3    |
|Jamaica Bay                                  |3    |
+---------------------------------------------+-----+
only showing top 5 rows

